In [5]:
!pip install -q chromadb

In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
!pip install sentence_transformers~=2.2.2

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm
import random
import json

import os

from transformers import LongformerModel, LongformerTokenizerFast
import collections
from transformers import GenerationConfig

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings


In [9]:
from sentence_transformers import SentenceTransformer

# Открытие Json файлов и преобазвание их в удобный вид


In [10]:
def open_json(file_name):
    with open(file_name, 'r', encoding="utf-8") as f:
        data = json.load(f)
    return data

In [11]:
text_data = open_json('parsed_data.json')
q_data = open_json('question.json')

In [33]:
data = []
titles = []
documents = {}
n = 1000
for i, value in enumerate(text_data):
    pre_data = []
    title = []
    for d in value['sections']:
        for j in range(0, len(d['content']), n):
            pre_data += [d['content'][j:j+n]]
            title += [(d['title'], value['url'])]
        titles += title
        data += pre_data
title_dict = dict()
title2idx = collections.defaultdict(list)
for i, t in enumerate(titles):
   title_dict[i] = t
   title2idx[t[1]] += [i]
# title_dict = {i: [t] for i, t in enumerate(titles)}
# title2idx = {t[1]: i for i, t in enumerate(titles)}


In [14]:
data_q = list(q_data.keys())
data_a = [a[0] for a in q_data.values()]
true_idx = [title2idx[a[1][0]] for a in q_data.values()]

# Создание collectin с помощью chroma

In [40]:
from transformers import AutoTokenizer, AutoModel


In [41]:
# #Definition of tokenizer and model for embendings
# model_embed = LongformerModel.from_pretrained('kazzand/ru-longformer-tiny-16384')
# tokenizer_longformer = LongformerTokenizerFast.from_pretrained('kazzand/ru-longformer-tiny-16384')
# e5_embed = SentenceTransformer('Nehc/e5-large-ru')
tokenizer_e5 = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model_e5 = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model):
        self.model = model
    def __call__(self, input: Documents) -> Embeddings:
        tokens_val = torch.tensor(self.model.encode(input))

        return tokens_val.tolist()


# creating embedding function
class BERTEmbeddingFunction(EmbeddingFunction):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
    def __call__(self, input: Documents) -> Embeddings:
        tokens_val = torch.tensor(self.tokenizer(input)['input_ids'])
        tokens_mask = torch.tensor(self.tokenizer(input)['attention_mask'])
        embeddings = self.model(tokens_val.long(), tokens_mask.long())['pooler_output']

        return embeddings.tolist()

# embedd_funct = MyEmbeddingFunction(model_embed)
bert_embedd_funct = BERTEmbeddingFunction(tokenizer_e5, model_e5)


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [42]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="colcol", embedding_function=bert_embedd_funct)

Загружаем необходимое количество файлов. (Можно приостановить при надобности)

In [44]:
for i, value in enumerate(tqdm(data)):
    collection.add(
        documents=[value],
        ids=str(i))

  0%|          | 0/20687 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Загрузка модели

In [ ]:
!pip install torch sentencepiece \
  accelerate \
  bitsandbytes \
  git+https://github.com/huggingface/transformers.git@15641892985b1d77acc74c9065c332cd7c3f7d7f \
  git+https://github.com/huggingface/peft.git

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_13b_lora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, MODEL_NAME)
model.eval()

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

In [46]:
def query_to_text(question, collection):
    d = collection.query(
        query_texts = [question],
        n_results=5
    )
    return f'''Контекст: {" ".join(d["documents"][0][:3])}\nВопрос: {question}\nОтвет:'''

In [50]:
question_ = "Какие меры предпринимает центральный банк для совершенствования своей политики и взаимодействия с обществом?"

In [49]:
data_val = tokenizer([query_to_text(question_, collection)], return_tensors="pt")
data_val = {k: v.to(model.device) for k, v in data_val.items() if k in ("input_ids", "attention_mask")}
output_ids = model.generate(**data_val, generation_config=generation_config)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

Контекст: и информационной открытости Банк России прежде всего стремится максимально оперативно и полно раскрывать информацию о целях, принципах, мерах и результатах денежно-кредитной политики, об оценке экономической ситуации и перспектив ее развития. Банк России также работает над расширением охвата коммуникации по денежно-кредитной политике и повышением ее адресности.

Подробнее, как Банк России рассказывает о своих решениях и информационной открытости Банк России прежде всего стремится максимально оперативно и полно раскрывать информацию о целях, принципах, мерах и результатах денежно-кредитной политики, об оценке экономической ситуации и перспектив ее развития. Банк России также работает над расширением охвата коммуникации по денежно-кредитной политике и повышением ее адресности.

Подробнее, как Банк России рассказывает о своих решениях и информационной открытости Банк России прежде всего стремится максимально оперативно и полно раскрывать информацию о целях, принципах, мерах и ре